In [1]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

import os 
import tensorflow as tf
from pathlib import Path
import cv2

# Run this before loading other dependencies, otherwise they might occupy memory on gpu 0 by default and it will stay that way

# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(device_count={'GPU': 1}, allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 1.0
tf.compat.v1.Session(config=config)

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet_v2 import ResNet101V2
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from src.architectures.simple.simple_base import SimpleBaseArchitecture
from src.architectures.adv.guendel19 import densenet
import numpy as np

from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from tensorflow.keras.optimizers import Adam, SGD
from src.architectures.benchmarks.benchmark_definitions import Chexpert_Benchmark, Chestxray14_Benchmark, simple_architecture_experiment, generate_benchmarks, METRICS, SINGLE_CLASS_METRICS, CHEXPERT_COLUMNS, CHESTXRAY14_COLUMNS
from src.metrics.metrics import F2Score
from src.metrics.losses import WeightedBinaryCrossentropy, BinaryCrossentropy, compute_class_weight


Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:04:00.0, compute capability: 7.5



In [2]:
columns_12 = ['Enlarged Cardiomediastinum',
                    'Cardiomegaly',
                    'Lung Opacity',
                    'Lung Lesion',
                    'Edema',
                    'Consolidation',
                    'Pneumonia',
                    'Atelectasis',
                    'Pneumothorax',
                    'Pleural Effusion',
                    'Pleural Other',
                    'Fracture']
columns_5 =  ['Cardiomegaly',
                'Edema',
                'Consolidation',
                'Atelectasis',
                'Pleural Effusion']

uzeros = ['Cardiomegaly',
        'Enlarged Cardiomediastinum',
        'Lung Opacity',
        'Lung Lesion',
        'Consolidation',
        'Pneumothorax',
        'Pleural Effusion']

uones = ['Edema',
        'Atelectasis',
        'Fracture',
        'Pleural Other',
        'Pneumonia',]

upsample_factors = {
    "Enlarged Cardiomediastinum": 1,
    "Lung Lesion":1,
    #"Pneumothorax":1,
    #"Pneumonia":1,
    "Pleural Other":2,
    "Fracture":2,
}
transformations_0 = {"hist_equalization":{}}
transformations_1 = { "gaussian_blur":{"kernal_size":3}, "hist_equalization":{}}
transformations_2 = {"unsharp_mask":{"radius":2, "amount":1}}
transformations_3 = {"windowing"}
transformations_4 = {}

In [3]:
bce_benchmark = Chexpert_Benchmark (path = Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY")),
                                             name="Chexpert_BCE_NoNorm_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled",
                                             classes=columns_12,
                                             train_labels = "train.csv",
                                             test_labels = "test.csv",
                                             nan_replacement = 0, #float("NaN"),
                                             u_enc = [uzeros, uones],
                                             epochs=5,
                                             batch_size=32,
                                             crop = False,
                                             dim=(256, 256),
                                             loss = BinaryCrossentropy(),
                                             use_class_weights = False,
                                             upsample_factors = upsample_factors,
                                             metrics=METRICS,
                                             single_class_metrics=SINGLE_CLASS_METRICS,
                                             optimizer = Adam(learning_rate=1e-4, clipnorm=1),
                                             lr_factor = 0.1,
                                             augmentation = "affine",
                                             transformations = transformations_4,
                                             split_seed = 6122156,
                                             split_valid_size = 0.05, 
                                             preprocess_input_fn = tf.keras.applications.xception.preprocess_input)


bce_benchmark.loss = WeightedBinaryCrossentropy(bce_benchmark.positive_weights,
                                                bce_benchmark.negative_weights)

Initialzed custom_binary_crossentropy with epsilon 1e-05


In [4]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = Xception(include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

prediction_layer = Dense(12, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=prediction_layer)
model.simple_name = "Xception-2"
bce_chexpert_exp = Experiment(bce_benchmark, model)

In [5]:
bce_chexpert_exp.train()

Epoch 1/5
8344/8344 [==============================] - ETA: 0s - loss: 0.2647 - auc: 0.7277 - precision: 0.6688 - recall: 0.0792 - f2_score: 0.0958 - binary_accuracy: 0.8278 - accuracy_enlarged_cardiomediastinum: 0.9035 - accuracy_cardiomegaly: 0.8878 - accuracy_lung_opacity: 0.5273 - accuracy_lung_lesion: 0.9175 - accuracy_edema: 0.7643 - accuracy_consolidation: 0.9335 - accuracy_pneumonia: 0.8921 - accuracy_atelectasis: 0.7085 - accuracy_pneumothorax: 0.9134 - accuracy_pleural_effusion: 0.6983 - accuracy_pleural_other: 0.9117 - accuracy_fracture: 0.8758 - auc_enlarged_cardiomediastinum: 0.6297 - auc_cardiomegaly: 0.7813 - auc_lung_opacity: 0.6896 - auc_lung_lesion: 0.7127 - auc_edema: 0.8128 - auc_consolidation: 0.6918 - auc_pneumonia: 0.6807 - auc_atelectasis: 0.6700 - auc_pneumothorax: 0.7669 - auc_pleural_effusion: 0.8258 - auc_pleural_other: 0.7688 - auc_fracture: 0.7020 - precision_enlarged_cardiomediastinum: 0.5064 - precision_cardiomegaly: 0.5545 - precision_lung_opacity: 0.58

In [6]:
bce_chexpert_exp.evaluate()

  1/234 [..............................] - ETA: 30s - loss: 0.1591 - auc: 0.0000e+00 - precision: 1.0000 - recall: 0.3333 - f2_score: 0.3846 - binary_accuracy: 0.8333 - accuracy_enlarged_cardiomediastinum: 1.0000 - accuracy_cardiomegaly: 1.0000 - accuracy_lung_opacity: 0.0000e+00 - accuracy_lung_lesion: 1.0000 - accuracy_edema: 1.0000 - accuracy_consolidation: 1.0000 - accuracy_pneumonia: 1.0000 - accuracy_atelectasis: 0.0000e+00 - accuracy_pneumothorax: 1.0000 - accuracy_pleural_effusion: 1.0000 - accuracy_pleural_other: 1.0000 - accuracy_fracture: 1.0000 - auc_enlarged_cardiomediastinum: 0.0000e+00 - auc_cardiomegaly: 0.0000e+00 - auc_lung_opacity: 0.0000e+00 - auc_lung_lesion: 0.0000e+00 - auc_edema: 0.0000e+00 - auc_consolidation: 0.0000e+00 - auc_pneumonia: 0.0000e+00 - auc_atelectasis: 0.0000e+00 - auc_pneumothorax: 0.0000e+00 - auc_pleural_effusion: 0.0000e+00 - auc_pleural_other: 0.0000e+00 - auc_fracture: 0.0000e+00 - precision_enlarged_cardiomediastinum: 0.0000e+00 - precisio

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1

234/234 [==============================] - 11s 49ms/step - loss: 0.3072 - auc: 0.7696 - precision: 0.6630 - recall: 0.1117 - f2_score: 0.1340 - binary_accuracy: 0.8162 - accuracy_enlarged_cardiomediastinum: 0.5385 - accuracy_cardiomegaly: 0.7350 - accuracy_lung_opacity: 0.4615 - accuracy_lung_lesion: 0.9829 - accuracy_edema: 0.8889 - accuracy_consolidation: 0.8675 - accuracy_pneumonia: 0.9573 - accuracy_atelectasis: 0.6667 - accuracy_pneumothorax: 0.9615 - accuracy_pleural_effusion: 0.7906 - accuracy_pleural_other: 0.9615 - accuracy_fracture: 0.9829 - auc_enlarged_cardiomediastinum: 0.5098 - auc_cardiomegaly: 0.7594 - auc_lung_opacity: 0.9020 - auc_lung_lesion: 0.9871 - auc_edema: 0.9303 - auc_consolidation: 0.9244 - auc_pneumonia: 0.7685 - auc_atelectasis: 0.8121 - auc_pneumothorax: 0.8496 - auc_pleural_effusion: 0.9190 - auc_pleural_other: 0.8734 - auc_fracture: 0.0000e+00 - precision_enlarged_cardiomediastinum: 0.6667 - precision_cardiomegaly: 0.8000 - precision_lung_opacity: 0.0000

{'report': '                            precision    recall  f1-score   support\n\nEnlarged Cardiomediastinum       0.33      0.01      0.02       109\n              Cardiomegaly       0.30      0.04      0.08        68\n              Lung Opacity       0.00      0.00      0.00       126\n               Lung Lesion       0.00      0.00      0.00         1\n                     Edema       0.15      0.09      0.11        45\n             Consolidation       0.00      0.00      0.00        33\n                 Pneumonia       0.00      0.00      0.00         8\n               Atelectasis       0.00      0.00      0.00        80\n              Pneumothorax       0.00      0.00      0.00         8\n          Pleural Effusion       0.41      0.13      0.20        67\n             Pleural Other       0.00      0.00      0.00         1\n                  Fracture       0.00      0.00      0.00         0\n\n                 micro avg       0.18      0.03      0.05       546\n                 m

In [7]:
bce_chexpert_exp.save()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /srv/idp-radio-1/models/Xception-2_Chexpert_BCE_NoNorm_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled/Xception-2_Chexpert_BCE_NoNorm_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled_20200826-201201/assets


'67131280-e7d8-11ea-b8fe-0242ac110005'

In [31]:
predictions = []

In [43]:
bce_benchmark = Chexpert_Benchmark (path = Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY")),
                                             name="Chexpert_BCE_NoNorm_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled",
                                             classes=columns_12,
                                             train_labels = "train.csv",
                                             test_labels = "test.csv",
                                             nan_replacement = 0, #float("NaN"),
                                             u_enc = [uzeros, uones],
                                             epochs=5,
                                             batch_size=32,
                                             crop = False,
                                             dim=(256, 256),
                                             loss = BinaryCrossentropy(),
                                             use_class_weights = False,
                                             upsample_factors = upsample_factors,
                                             metrics=METRICS,
                                             single_class_metrics=SINGLE_CLASS_METRICS,
                                             optimizer = Adam(learning_rate=1e-4, clipnorm=1),
                                             lr_factor = 0.1,
                                             augmentation = "affine",
                                             transformations = transformations_4,
                                             split_seed = 6122156,
                                             split_valid_size = 0.05, 
                                             preprocess_input_fn = tf.keras.applications.densenet.preprocess_input)

In [47]:
from tensorflow.keras.applications.densenet import DenseNet169
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = DenseNet169(include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

prediction_layer = Dense(12, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=prediction_layer)
model.simple_name = "DenseNet121-2"
bce_chexpert_exp = Experiment(bce_benchmark, model)

In [48]:
bce_chexpert_exp.model.load_weights("models/DenseNet169_Chexpert_BCE_NoNorm_E5_B32_C0_N12_AugAffine_sharp21_U75_D256_DS9505_1LR4_LF1_Adam_Upsampled/weights.05-0.21.hdf5")
predictions.append(bce_chexpert_exp.model.predict(bce_chexpert_exp.benchmark.testgen))

In [49]:
len(predictions)

4

In [57]:
for x in predictions:
    print(len(x[0]))

12
12
12
12


In [59]:
from sklearn.metrics import classification_report
y_pred = tf.math.reduce_mean(predictions, axis=0)
predictions_bool = (y_pred >= 0.5)
y_pred = np.array(predictions_bool, dtype=int)
groundtruth_label = bce_chexpert_exp.benchmark.testgen.get_labels_nonan()
report = classification_report(groundtruth_label, y_pred, target_names=bce_chexpert_exp.benchmark.label_columns)
print('skelarn report: ', report)

skelarn report:                              precision    recall  f1-score   support

Enlarged Cardiomediastinum       0.00      0.00      0.00       109
              Cardiomegaly       0.00      0.00      0.00        68
              Lung Opacity       0.00      0.00      0.00       126
               Lung Lesion       0.00      0.00      0.00         1
                     Edema       0.33      0.02      0.04        45
             Consolidation       0.00      0.00      0.00        33
                 Pneumonia       0.00      0.00      0.00         8
               Atelectasis       0.00      0.00      0.00        80
              Pneumothorax       0.00      0.00      0.00         8
          Pleural Effusion       0.00      0.00      0.00        67
             Pleural Other       0.00      0.00      0.00         1
                  Fracture       0.00      0.00      0.00         0

                 micro avg       0.33      0.00      0.00       546
                 macro avg   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1